# Notebook for the flu ensemble modeling challenge

### Load dependencies

In [1]:
import os
import json
import sympy
import torch
import numpy as np
import pandas as pd
from typing import Dict, List, Callable

# MIRA modeling dependencies
from mira.metamodel import *
from mira.metamodel.ops import stratify
from mira.examples.concepts import susceptible, exposed, infected, recovered
from mira.modeling import Model
from mira.modeling.amr.petrinet import AMRPetriNetModel, template_model_to_petrinet_json
from mira.sources.amr.petrinet import template_model_from_amr_json
from mira.metamodel.io import model_to_json_file, model_from_json_file

# PyCIEMSS dependencies
import pyciemss
import pyciemss.visuals.plots as plots
import pyciemss.visuals.vega as vega
import pyciemss.visuals.trajectories as trajectories
from pyciemss.integration_utils.intervention_builder import (
    param_value_objective,
    start_time_objective,
)

# Process data

In [5]:
data22_23 = pd.read_csv("FL-PA-Truth-22-23.csv")
data23_24 = pd.read_csv("FL-PA-Truth-23-24.csv")

## Set Florida and Pennsylvania population parameters and initial values

In [2]:
FL_total_population = 23_370_000
PA_total_population = 13_080_000

# Build model 1

### Define units

In [ ]:
# TODO: change to weekly
person_units = lambda: Unit(expression=sympy.Symbol('person'))
day_units = lambda: Unit(expression=sympy.Symbol('day'))
per_day_units = lambda: Unit(expression=1/sympy.Symbol('day'))
dimensionless_units = lambda: Unit(expression=sympy.Integer('1'))
per_day_per_person_units = lambda: Unit(expression=1/(sympy.Symbol('day')*sympy.Symbol('person')))

### Define and stratify model concepts

In [ ]:
_susceptible = Concept(name='S', units=person_units(), identifiers={'ido': '0000514'})
_exposed = Concept(name='E', units=person_units(), identifiers={'apollosv': '00000154'})
_infected = Concept(name='I', units=person_units(), identifiers={'ido': '0000511'})

c = {
    'S_u': _susceptible.with_context(status="unvaccinated"),
    'S_v': _susceptible.with_context(status="vaccinated"),
    
    'E_u': _exposed.with_context(status="unvaccinated"),
    'E_v': _exposed.with_context(status="vaccinated"),
    
    'I_u': _infected.with_context(status="unvaccinated"),
    'I_v': _infected.with_context(status="vaccinated"),
    
    'R': Concept(name='R', units=person_units(), identifiers={'ido': '0000592'}),
    'H': Concept(name="H", units=person_units(), identifiers={"ido": "0000511"},
                        context={"property": "ncit:C25179"}),
    'D': Concept(name="D", units=person_units(), identifiers={"ncit": "C28554"}),
    
    "Cumulative_cases": Concept(name="Cumulative_cases", units=person_units()),
}

for concept in c:
    c[concept].name = concept

### Define parameter values and uncertainty

In [ ]:

parameters = {
    'beta': Parameter(name='beta', value=sympy.Float(0.125), units=per_day_units(),
                      distribution=Distribution(type='StandardUniform1',
                                                parameters={'minimum': 0.12,
                                                            'maximum': 0.135})),  # Transmission rate

    'NPI_mult': Parameter(name='NPI_mult', value=sympy.Float(1.0), units=dimensionless_units(),
                      distribution=Distribution(type='StandardUniform1',
                                                parameters={'minimum': 0.999,
                                                            'maximum': 1.001})),  # NPI (social distancing/masking) multiplier

    'vacc_mult': Parameter(name='vacc_mult', value=sympy.Float(vacc_effectiveness), units=dimensionless_units(),
                      distribution=None),  # Vaccination multiplier
    
    'N': Parameter(name='total_population', value=sympy.Float(LA_county_total_pop), units=person_units()),  # Total population
    
    'r_EI': Parameter(name='r_EI', value=sympy.Float(0.1424), units=per_day_units(),
                     distribution=Distribution(type='StandardUniform1',
                                                parameters={'minimum': 0.1,
                                                            'maximum': 0.25})),  # Rate of progressing E -> I
    
    'r_IR_u': Parameter(name='r_IR_u', value=sympy.Float(0.15), units=per_day_units(),
                     distribution=Distribution(type='StandardUniform1',
                                                parameters={'minimum': 0.1,
                                                            'maximum': 0.2})),  # Rate of progressing I_u -> R
    'r_IR_v': Parameter(name='r_IR_v', value=sympy.Float(0.162), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                            parameters={'minimum': 0.15,
                                                        'maximum': 0.2})),  # Rate of progressing I_v -> R

    'r_IH_u': Parameter(name='r_IH_u', value=sympy.Float(0.0041), units=per_day_units(),
                     distribution=Distribution(type='StandardUniform1',
                                                parameters={'minimum': 0.004,
                                                            'maximum': 0.0045})),  # Rate of progressing I_u -> H
    'r_IH_v': Parameter(name='r_IH_v', value=sympy.Float(0.0013), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                                parameters={'minimum': 0.001,
                                                            'maximum': 0.0015})),  # Rate of progressing I_v -> H
    
    'r_HR': Parameter(name='r_HR', value=sympy.Float(0.155), units=per_day_units(),
                     distribution=Distribution(type='StandardUniform1',
                                                parameters={'minimum': 0.12,
                                                            'maximum': 0.2})),  # Rate of progressing H -> R
    
    'r_HD': Parameter(name='r_HD', value=sympy.Float(0.0099), units=per_day_units(),
                     distribution=Distribution(type='StandardUniform1',
                                                parameters={'minimum': 0.0095,
                                                            'maximum': 0.012})),  # Rate of progressing H -> D

    'r_Sv': Parameter(name='r_Sv', value=sympy.Float(10000), units=per_day_units(),
                     distribution=Distribution(type='StandardUniform1',
                                                parameters={'minimum': 9999,
                                                            'maximum': 10001})),  # number of R and Su vaccinated per day
    
    'r_SvSu': Parameter(name='r_SvSu', value=sympy.Float(0.002), units=per_day_units(),
                     distribution=None),  # Vaccination duration = 1 year
}

### Define `SymPy` variables

In [6]:
S_u, S_v, E_u, E_v, I_u, I_v, R, H, D, Cumulative_cases, beta, NPI_mult, vacc_mult, N, r_EI, r_IR_u, r_IR_v, r_IH_u, r_IH_v, r_HR, r_HD, r_Sv, r_SvSu = \
    sympy.symbols(
        'S_u S_v E_u E_v I_u I_v R H D Cumulative_cases beta NPI_mult vacc_mult N r_EI r_IR_u r_IR_v r_IH_u r_IH_v r_HR r_HD r_Sv r_SvSu'
    )

### Set initial conditions

In [ ]:
initials = {
    "S_u": Initial(concept=c["S_u"], expression=initial_susceptible*(1 - p_vacc)),
    "S_v": Initial(concept=c["S_v"], expression=initial_susceptible*p_vacc),
    "E_u": Initial(concept=c["E_u"], expression=2*initial_exposed/3),
    "E_v": Initial(concept=c["E_v"], expression=initial_exposed/3),
    "I_u": Initial(concept=c["I_u"], expression=2*initial_cases/3),
    "I_v": Initial(concept=c["I_v"], expression=initial_cases/3),
    "R": Initial(concept=c["R"], expression=initial_recovered),
    'H': Initial(concept=c["H"], expression=initial_hosp),
    'D': Initial(concept=c["D"], expression=initial_deaths),
    "Cumulative_cases": Initial(concept=c["Cumulative_cases"], expression=initial_cumulative_cases),
}

### Define model templates

In [8]:
##### S -> E
# Su -> Eu by Iu
sueuiu = ControlledConversion(
    subject=c['S_u'],
    outcome=c['E_u'],
    controller=c['I_u'],
    rate_law=beta*NPI_mult*S_u*I_u / N
)
# Sv -> Ev by Iu
sveviu = ControlledConversion(
    subject=c['S_v'],
    outcome=c['E_v'],
    controller=c['I_u'],
    rate_law=beta*NPI_mult*vacc_mult*S_v*I_u / N
)
# Su -> Eu by Iv
sueuiv = ControlledConversion(
    subject=c['S_u'],
    outcome=c['E_u'],
    controller=c['I_v'],
    rate_law=beta*NPI_mult*S_u*I_v / N
)
# Sv -> Ev by Iv
sveviv = ControlledConversion(
    subject=c['S_v'],
    outcome=c['E_v'],
    controller=c['I_v'],
    rate_law=beta*NPI_mult*vacc_mult*S_v*I_v / N
)


##### Vaccination
# Su -> Sv
susv = NaturalConversion(
    subject=c['S_u'],
    outcome=c['S_v'],
    rate_law=r_Sv
)
# # R -> Sv
# rsv = NaturalConversion(
#     subject=c['R'],
#     outcome=c['S_v'],
#     rate_law=r_Sv
# )
# Sv -> Su
svsu = NaturalConversion(
    subject=c['S_v'],
    outcome=c['S_u'],
    rate_law=r_SvSu*S_v
)


#### E -> I
# Eu -> Iu
euiu = NaturalConversion(
    subject=c['E_u'],
    outcome=c['I_u'],
    rate_law=r_EI*E_u
)
# Ev -> Iv
eviv = NaturalConversion(
    subject=c['E_v'],
    outcome=c['I_v'],
    rate_law=r_EI*E_v
)


#### I -> R
# Iu -> R
iuru = NaturalConversion(
    subject=c['I_u'],
    outcome=c['R'],
    rate_law=r_IR_u*I_u
)
# Iv -> R
ivrv = NaturalConversion(
    subject=c['I_v'],
    outcome=c['R'],
    rate_law=r_IR_v*I_v
)


#### I -> H
# Iu -> H
iuhu = NaturalConversion(
    subject=c['I_u'],
    outcome=c['H'],
    rate_law=r_IH_u*I_u
)
# Iv -> H
ivhv = NaturalConversion(
    subject=c['I_v'],
    outcome=c['H'],
    rate_law=r_IH_v*I_v
)


#### H -> R
# H -> R
hr = NaturalConversion(
    subject=c['H'],
    outcome=c['R'],
    rate_law=r_HR*H
)


#### H -> D
# Hu -> Du
hd = NaturalConversion(
    subject=c['H'],
    outcome=c['D'],
    rate_law=r_HD*H
)


### Cumulative Cases
ccases = GroupedControlledProduction(
    controllers=[c['E_u'], c['E_v']],
    outcome=c['Cumulative_cases'],
    rate_law=r_EI*(E_u + E_v)
)

NameError: name 'c' is not defined

### Define observables

In [ ]:
observables_seir = {
    'cases': Observable(name='cases', expression=I_u+I_v),
    'hospitalized': Observable(name='hospitalized', expression=H),
    'deceased': Observable(name='deceased', expression=D),
    'cumulative_cases': Observable(name='cumulative_cases', expression=Cumulative_cases),
}

### Define template model and save as petrinet AMR

In [ ]:
# TODO: change model name
seir_model = TemplateModel(
    templates=[
        sueuiu,
        sveviu,
        sueuiv,
        sveviv,
        susv,
        # rsv,
        svsu,
        euiu,
        eviv,
        iuhu,
        ivhv,
        hr,
        hd,
        ccases
    ],
    parameters=parameters,
    initials=initials,
    time=Time(name='t', units=day_units()),
    observables=observables_seir,
    annotations=Annotations(name='SEIRHD vacc model for LA County t0 = 10/28/2021')
)

# Save as JSON
with open("SEIRHD_vacc_petrinet.json", 'w') as fh:
    json.dump(template_model_to_petrinet_json(seir_model), fh, indent=1)

# Simulate the model

### Set parameters for sampling

In [ ]:
model = "SEIRHD_vacc_petrinet.json"

start_time = 0.0
end_time = 151.0 # simulate from 10/28/2021 to 03/28/2022
logging_step_size = 1.0
num_samples = 100

In [ ]:
### Simulate model 1

In [ ]:
result1 = pyciemss.sample(model1, end_time, logging_step_size, num_samples, start_time=start_time)
display(result1['data'].head())

# Plot results for all states
schema = plots.trajectories(result1["data"], keep=".*_state")
plots.save_schema(schema, "_schema.json")
plots.ipy_display(schema, dpi=150)

### Set up and perform calibration

In [ ]:
data_mapping = {"cases": "cumulative_cases", "deaths": "deceased"} # data is mapped to observables

num_iterations = 500
calibrated_results = pyciemss.calibrate(
    model, 
    data, 
    data_mapping=data_mapping, 
    num_iterations=num_iterations
)
parameter_estimates = calibrated_results["inferred_parameters"]
parameter_estimates()

### Simulate a 3-month forecast and plot results with data (Scenario 2 - Part I)

In [ ]:
calibrated_sample_results = pyciemss.sample(
    model, 
    end_time, 
    logging_step_size, 
    num_samples,             
    start_time=start_time, 
    inferred_parameters=parameter_estimates
)
# display(calibrated_sample_results["data"].head())

# Plot the result
nice_labels = {
        "cumulative_cases_observable_state": "Cumulative Cases",
        "deceased_observable_state": "Deaths", 
        }
nice_data_names = {
        "cases": "Case Data",
        "deaths": "Death Data"
        }
data_df = pd.read_csv(data)
data_df.rename(columns=nice_data_names, inplace=True)
schema = plots.trajectories(
    pd.DataFrame(calibrated_sample_results["data"]), 
    keep=["cumulative_cases_observable_state", "deceased_observable_state"], 
    relabel=nice_labels,
    points=data_df.drop(columns=['Timestamp']).reset_index(drop=True)
)

plots.save_schema(schema, "_schema.json")
plots.ipy_display(schema, dpi=150)